# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import geoviews


# Import API key
from api_keys import geoapify_key

D:\anaconda\envs\dev\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,jamestown,42.0970,-79.2353,8.90,98,100,2.57,US,1698638283
1,1,vorgashor,67.5833,63.9500,-19.07,99,38,2.04,RU,1698638480
2,2,whistler,50.1164,-122.9695,1.42,78,74,0.96,CA,1698638481
3,3,dauphin,51.1494,-100.0502,-3.92,86,100,3.09,CA,1698638481
4,4,georgetown,5.4112,100.3354,29.99,74,20,4.12,MY,1698638480


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

humidity = city_data_df["Humidity"].astype(int)

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75
)


# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values

ideal_weather_city_df = city_data_df[(city_data_df['Cloudiness'] <=10.0) & (city_data_df['Wind Speed'] <= 10.0) &\
                        (city_data_df['Humidity'] >= 60) & (city_data_df['Humidity'] < 100) \
                                     & (city_data_df['Max Temp'] <= 30) & (city_data_df['Max Temp'] >22)]

# Drop any rows with null values
city_data_clean_df = ideal_weather_city_df.dropna()

# Display sample data
city_data_clean_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,tura,25.5198,90.2201,26.41,65,1,0.60,IN,1698638484
59,59,ormara,25.2088,64.6357,26.95,71,3,2.09,PK,1698638472
119,119,koungheul,13.9833,-14.8000,25.09,81,7,2.61,SN,1698638538
148,148,kailua-kona,19.6406,-155.9956,29.04,63,1,0.45,US,1698638509
246,246,howrah,22.5892,88.3103,28.00,65,3,2.06,IN,1698638600
261,261,meraux,29.9330,-89.9498,24.00,89,0,3.09,US,1698638608
264,264,saint-louis,16.3333,-15.0000,24.80,61,0,3.56,SN,1698638610
269,269,cabo san lucas,22.8909,-109.9124,26.40,75,0,3.09,MX,1698638401
365,365,aswan,24.0934,32.9070,26.61,69,0,2.57,EG,1698638660
393,393,saint-pierre,-21.3393,55.4781,26.98,65,10,1.54,RE,1698638675


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_clean_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,tura,IN,25.5198,90.2201,65,
59,ormara,PK,25.2088,64.6357,71,
119,koungheul,SN,13.9833,-14.8000,81,
148,kailua-kona,US,19.6406,-155.9956,63,
246,howrah,IN,22.5892,88.3103,65,
261,meraux,US,29.9330,-89.9498,89,
264,saint-louis,SN,16.3333,-15.0000,61,
269,cabo san lucas,MX,22.8909,-109.9124,75,
365,aswan,EG,24.0934,32.9070,69,
393,saint-pierre,RE,-21.3393,55.4781,65,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
categories = 'accommodation.hotel'
radius = 10000

params = {
    "categories":categories,
    "limit":1,
    "lang":'en',
    "apiKey":geoapify_key  
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']

    # Add parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places?"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
ormara - nearest hotel: No hotel found
koungheul - nearest hotel: Auberge Keur Ghan
kailua-kona - nearest hotel: Kona Seaside Hotel
howrah - nearest hotel: Sun India Guest House
meraux - nearest hotel: Holiday Inn Express & Suites Chalmette - New Orleans S
saint-louis - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
aswan - nearest hotel: Yaseen hotel
saint-pierre - nearest hotel: Tropic Hotel
miramar - nearest hotel: El Palacio Sports Hotel
holualoa - nearest hotel: Kona Hotel
uturoa - nearest hotel: Hawaiki Nui hotel
maintirano - nearest hotel: Hotel Safari
gafargaon - nearest hotel: Touhid e chalet তৌহিদ ই কাষ্ঠনির্মিত কুটীর
yanbu - nearest hotel: No hotel found
kaliganj - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
8,tura,IN,25.5198,90.2201,65,No hotel found
59,ormara,PK,25.2088,64.6357,71,No hotel found
119,koungheul,SN,13.9833,-14.8000,81,Auberge Keur Ghan
148,kailua-kona,US,19.6406,-155.9956,63,Kona Seaside Hotel
246,howrah,IN,22.5892,88.3103,65,Sun India Guest House
261,meraux,US,29.9330,-89.9498,89,Holiday Inn Express & Suites Chalmette - New O...
264,saint-louis,SN,16.3333,-15.0000,61,No hotel found
269,cabo san lucas,MX,22.8909,-109.9124,75,Comfort Rooms
365,aswan,EG,24.0934,32.9070,69,Yaseen hotel
393,saint-pierre,RE,-21.3393,55.4781,65,Tropic Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
# %%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 750,
    frame_height = 600,
    size = 60.0,
    scale = 0.5,
    color = "City",
    hover_cols=['Hotel Name', 'Country']
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)